# Pipeline of Digits

This is a starting notebook for solving the "Pipeline of Digits" assignment.


This notebook was created by [Santiago L. Valdarrama](https://twitter.com/svpino) as part of the [Machine Learning School](https://www.ml.school) program.

Let's make sure we are running the latest version of the SakeMaker's SDK. **Restart the notebook** after you upgrade the library.

In [4]:
!pip install -q --upgrade pip
!pip install -q --upgrade awscli boto3
!pip install -q --upgrade PyYAML==6.0
!pip install -q --upgrade sagemaker==2.165.0
!pip show sagemaker

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at 

In [68]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
import boto3
import sagemaker
import pandas as pd

import sys
from pathlib import Path

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()

## Creating the S3 Bucket

Let's create an S3 bucket where you will upload all the information generated by the pipeline. Make sure you set `BUCKET` to the name of the bucket you want to use. This name has to be unique.

If you want to create a bucket in a region other than `us-east-1`, use this command instead:

```
!aws s3api create-bucket --bucket $BUCKET --create-bucket-configuration LocationConstraint=$region
```

The `LocationConstraint` argument should specify the region where you want to create the bucket.

In [70]:
BUCKET = "mlschool-cda"

!aws s3api create-bucket --bucket $BUCKET

{
    "Location": "/mlschool-cda"
}


## Loading the dataset

We have two CSV files containing the MNIST dataset. These files come from the [MNIST in CSV](https://www.kaggle.com/datasets/oddrationale/mnist-in-csv) Kaggle dataset.

The `mnist_train.csv` file contains 60,000 training examples and labels. The `mnist_test.csv` contains 10,000 test examples and labels. Each row consists of 785 values: the first value is the label (a number from 0 to 9) and the remaining 784 values are the pixel values (a number from 0 to 255).

Let's extract the `dataset.tar.gz` file.

In [71]:
MNIST_FOLDER = "mnist"
DATASET_FOLDER = Path("dataset")

!tar -xvzf dataset.tar.gz --no-same-owner

dataset/
dataset/mnist_test.csv
dataset/mnist_train.csv


Let's load the first 10 rows of the test set.

In [72]:
df_train = pd.read_csv(DATASET_FOLDER / "mnist_train.csv", nrows=10)
df_train

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
df_test = pd.read_csv(DATASET_FOLDER / "mnist_test.csv", nrows=10)
df_test

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Uploading dataset to S3

In [74]:
S3_FILEPATH = f"s3://{BUCKET}/{MNIST_FOLDER}"


TRAIN_SET_S3_URI = sagemaker.s3.S3Uploader.upload(
    local_path=str(DATASET_FOLDER / "mnist_train.csv"), 
    desired_s3_uri=S3_FILEPATH,
)

TEST_SET_S3_URI = sagemaker.s3.S3Uploader.upload(
    local_path=str(DATASET_FOLDER / "mnist_test.csv"), 
    desired_s3_uri=S3_FILEPATH,
)

print(f"Train set S3 location: {TRAIN_SET_S3_URI}")
print(f"Test set S3 location: {TEST_SET_S3_URI}")

Train set S3 location: s3://mlschool-cda/mnist/mnist_train.csv
Test set S3 location: s3://mlschool-cda/mnist/mnist_test.csv


### Create a CODE_FOLDER_MNIST

In [75]:
CODE_FOLDER_MNIST = Path("codemnist")
CODE_FOLDER_MNIST.mkdir(parents=True, exist_ok=True)

sys.path.append(f"./{CODE_FOLDER_MNIST}")

## Lifecycle Configuration

In [76]:
%%writefile packagesmnist.sh

#!/bin/bash
# This script upgrades the packages on a SageMaker 
# Studio Kernel Application.

set -eux

pip install -q --upgrade pip
pip install -q --upgrade awscli boto3
pip install -q --upgrade scikit-learn==0.23.2
pip install -q --upgrade PyYAML==6.0
pip install -q --upgrade sagemaker

Overwriting packagesmnist.sh


### Sagemaker Domain and User ID

In [77]:
DOMAIN_ID = "d-aznapotw0gcd"
USER_PROFILE = "mhrecaldeb-1690232830133"

In [78]:
import sagemaker

role = sagemaker.get_execution_role()
role

'arn:aws:iam::759575403360:role/service-role/AmazonSageMaker-ExecutionRole-20230724T160653'

## Configurar los permisos segun instrucciones

Open the Amazon IAM service, find the role and edit the custom Execution Policy assigned to it. You can edit the permissions of the Execution Policy and use the following definition instead:

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "IAM0",
            "Effect": "Allow",
            "Action": [
                "iam:CreateServiceLinkedRole"
            ],
            "Resource": "*",
            "Condition": {
                "StringEquals": {
                    "iam:AWSServiceName": [
                        "autoscaling.amazonaws.com",
                        "ec2scheduled.amazonaws.com",
                        "elasticloadbalancing.amazonaws.com",
                        "spot.amazonaws.com",
                        "spotfleet.amazonaws.com",
                        "transitgateway.amazonaws.com"
                    ]
                }
            }
        },
        {
            "Sid": "IAM1",
            "Effect": "Allow",
            "Action": [
                "iam:CreateRole",
                "iam:PassRole",
                "iam:AttachRolePolicy"
            ],
            "Resource": "*"
        },
        {
            "Sid": "Lambda",
            "Effect": "Allow",
            "Action": [
                "lambda:CreateFunction",
                "lambda:DeleteFunction",
                "lambda:InvokeFunctionUrl",
                "lambda:InvokeFunction",
                "lambda:UpdateFunctionCode",
                "lambda:InvokeAsync"
            ],
            "Resource": "*"
        },
        {
            "Sid": "SageMaker",
            "Effect": "Allow",
            "Action": [
                "sagemaker:UpdateDomain",
                "sagemaker:UpdateUserProfile"
            ],
            "Resource": "*"
        },
        {
            "Sid": "CloudWatch",
            "Effect": "Allow",
            "Action": [
                "cloudwatch:PutMetricData",
                "cloudwatch:GetMetricData",
                "cloudwatch:DescribeAlarmsForMetric",
                "logs:CreateLogStream",
                "logs:PutLogEvents",
                "logs:CreateLogGroup",
                "logs:DescribeLogStreams"
            ],
            "Resource": "*"
        },
        {
            "Sid": "ECR",
            "Effect": "Allow",
            "Action": [
                "ecr:GetAuthorizationToken",
                "ecr:BatchCheckLayerAvailability",
                "ecr:GetDownloadUrlForLayer",
                "ecr:BatchGetImage"
            ],
            "Resource": "*"
        },
        {
            "Sid": "S3",
            "Effect": "Allow",
            "Action": [
                "s3:CreateBucket",
                "s3:ListBucket",
                "s3:GetBucketLocation",
                "s3:PutObject",
                "s3:GetObject",
                "s3:DeleteObject"
            ],
            "Resource": "arn:aws:s3:::*"
        }
    ]
}
```

## Lifecycle Configuration for packages

In [79]:
%%bash -s "$DOMAIN_ID" "$USER_PROFILE"

DOMAIN_ID=$(echo "$1")
USER_PROFILE=$(echo "$2")

LCC_CONTENT=`openssl base64 -A -in packagesmnist.sh`

aws sagemaker delete-studio-lifecycle-config \
    --studio-lifecycle-config-name packagesmnist

response=$(aws sagemaker create-studio-lifecycle-config \
    --studio-lifecycle-config-name packagesmnist \
    --studio-lifecycle-config-content $LCC_CONTENT \
    --studio-lifecycle-config-app-type KernelGateway) 

arn=$(echo "${response}" | python3 -c "import sys, json; print(json.load(sys.stdin)['StudioLifecycleConfigArn'])")
echo "${arn}"

aws sagemaker update-user-profile --domain-id $DOMAIN_ID \
    --user-profile-name $USER_PROFILE \
    --user-settings '{
        "KernelGatewayAppSettings": {
            "LifecycleConfigArns": ["'$arn'"]
        }
    }'


An error occurred (ResourceInUse) when calling the DeleteStudioLifecycleConfig operation: Unable to delete LCC [arn:aws:sagemaker:us-east-1:759575403360:studio-lifecycle-config/packagesmnist] because LCC is in use by one or more Apps

An error occurred (ResourceInUse) when calling the CreateStudioLifecycleConfig operation: The ID or Name specified is already in use.
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/conda/lib/python3.7/json/__init__.py", line 296, in load
    parse_constant=parse_constant, object_pairs_hook=object_pairs_hook, **kw)
  File "/opt/conda/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/opt/conda/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/conda/lib/python3.7/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: 

CalledProcessError: Command 'b'\nDOMAIN_ID=$(echo "$1")\nUSER_PROFILE=$(echo "$2")\n\nLCC_CONTENT=`openssl base64 -A -in packagesmnist.sh`\n\naws sagemaker delete-studio-lifecycle-config \\\n    --studio-lifecycle-config-name packagesmnist\n\nresponse=$(aws sagemaker create-studio-lifecycle-config \\\n    --studio-lifecycle-config-name packagesmnist \\\n    --studio-lifecycle-config-content $LCC_CONTENT \\\n    --studio-lifecycle-config-app-type KernelGateway) \n\narn=$(echo "${response}" | python3 -c "import sys, json; print(json.load(sys.stdin)[\'StudioLifecycleConfigArn\'])")\necho "${arn}"\n\naws sagemaker update-user-profile --domain-id $DOMAIN_ID \\\n    --user-profile-name $USER_PROFILE \\\n    --user-settings \'{\n        "KernelGatewayAppSettings": {\n            "LifecycleConfigArns": ["\'$arn\'"]\n        }\n    }\'\n'' returned non-zero exit status 255.

## Code for autoshutdown

In [80]:
%%writefile autoshutdownmnist.sh

#!/bin/bash
# This script installs the idle notebook auto-checker server extension to SageMaker Studio
# The original extension has a lab extension part where users can set the idle timeout via a Jupyter Lab widget.
# In this version the script installs the server side of the extension only. The idle timeout
# can be set via a command-line script which will be also created by this create and places into the
# user's home folder
#
# Installing the server side extension does not require Internet connection (as all the dependencies are stored in the
# install tarball) and can be done via VPCOnly mode.

set -eux

# timeout in minutes
export TIMEOUT_IN_MINS=60

# Should already be running in user home directory, but just to check:
cd /home/sagemaker-user

# By working in a directory starting with ".", we won't clutter up users' Jupyter file tree views
mkdir -p .auto-shutdown

# Create the command-line script for setting the idle timeout
cat > .auto-shutdown/set-time-interval.sh << EOF
#!/opt/conda/bin/python
import json
import requests
TIMEOUT=${TIMEOUT_IN_MINS}
session = requests.Session()
# Getting the xsrf token first from Jupyter Server
response = session.get("http://localhost:8888/jupyter/default/tree")
# calls the idle_checker extension's interface to set the timeout value
response = session.post("http://localhost:8888/jupyter/default/sagemaker-studio-autoshutdown/idle_checker",
            json={"idle_time": TIMEOUT, "keep_terminals": False},
            params={"_xsrf": response.headers['Set-Cookie'].split(";")[0].split("=")[1]})
if response.status_code == 200:
    print("Succeeded, idle timeout set to {} minutes".format(TIMEOUT))
else:
    print("Error!")
    print(response.status_code)
EOF
chmod +x .auto-shutdown/set-time-interval.sh

# "wget" is not part of the base Jupyter Server image, you need to install it first if needed to download the tarball
sudo yum install -y wget
# You can download the tarball from GitHub or alternatively, if you're using VPCOnly mode, you can host on S3
wget -O .auto-shutdown/extension.tar.gz https://github.com/aws-samples/sagemaker-studio-auto-shutdown-extension/raw/main/sagemaker_studio_autoshutdown-0.1.5.tar.gz

# Or instead, could serve the tarball from an S3 bucket in which case "wget" would not be needed:
# aws s3 --endpoint-url [S3 Interface Endpoint] cp s3://[tarball location] .auto-shutdown/extension.tar.gz

# Installs the extension
cd .auto-shutdown
tar xzf extension.tar.gz
cd sagemaker_studio_autoshutdown-0.1.5

# Activate studio environment just for installing extension
export AWS_SAGEMAKER_JUPYTERSERVER_IMAGE="${AWS_SAGEMAKER_JUPYTERSERVER_IMAGE:-'jupyter-server'}"
if [ "$AWS_SAGEMAKER_JUPYTERSERVER_IMAGE" = "jupyter-server-3" ] ; then
    eval "$(conda shell.bash hook)"
    conda activate studio
fi;
pip install --no-dependencies --no-build-isolation -e .
jupyter serverextension enable --py sagemaker_studio_autoshutdown
if [ "$AWS_SAGEMAKER_JUPYTERSERVER_IMAGE" = "jupyter-server-3" ] ; then
    conda deactivate
fi;

# Restarts the jupyter server
nohup supervisorctl -c /etc/supervisor/conf.d/supervisord.conf restart jupyterlabserver

# Waiting for 30 seconds to make sure the Jupyter Server is up and running
sleep 30

# Calling the script to set the idle-timeout and active the extension
/home/sagemaker-user/.auto-shutdown/set-time-interval.sh

Overwriting autoshutdownmnist.sh


## Lifecycle config with autoshutdown

In [81]:
%%bash -s "$DOMAIN_ID" "$USER_PROFILE"

DOMAIN_ID=$(echo "$1")
USER_PROFILE=$(echo "$2")

LCC_CONTENT=`openssl base64 -A -in autoshutdownmnist.sh`

aws sagemaker delete-studio-lifecycle-config \
    --studio-lifecycle-config-name autoshutdownmnist 2> /dev/null

response=$(aws sagemaker create-studio-lifecycle-config \
    --studio-lifecycle-config-name autoshutdownmnist \
    --studio-lifecycle-config-content $LCC_CONTENT \
    --studio-lifecycle-config-app-type JupyterServer) 

arn=$(echo "${response}" | python3 -c "import sys, json; print(json.load(sys.stdin)['StudioLifecycleConfigArn'])")
echo "${arn}"

aws sagemaker update-user-profile --domain-id $DOMAIN_ID \
    --user-profile-name $USER_PROFILE \
    --user-settings '{
        "JupyterServerAppSettings": {
            "DefaultResourceSpec": {
                "LifecycleConfigArn": "'$arn'",
                "InstanceType": "system"
            },
            "LifecycleConfigArns": ["'$arn'"]
        }
    }'

arn:aws:sagemaker:us-east-1:759575403360:studio-lifecycle-config/autoshutdownmnist
{
    "UserProfileArn": "arn:aws:sagemaker:us-east-1:759575403360:user-profile/d-aznapotw0gcd/mhrecaldeb-1690232830133"
}


## Constants

In [82]:
%%writefile {CODE_FOLDER_MNIST}/constantsmnist.py

import boto3
import sagemaker
import sys
from pathlib import Path

BUCKET = "mlschool-cda"
MNIST_FOLDER = "mnist"


S3_FILEPATH = f"s3://{BUCKET}/{MNIST_FOLDER}"

# DATASET_FOLDER = Path("dataset")

DATA_FILEPATH_TRAIN = Path().resolve() / "dataset" / "mnist_train.csv" #estas variables estan definidas aqui
DATA_FILEPATH_TEST = Path().resolve() / "dataset" / "mnist_test.csv"

#*************************************************************
#TRAIN_SET_S3_URI = sagemaker.s3.S3Uploader.upload(
#    local_path=str(DATASET_FOLDER / "mnist_train.csv"), 
#    desired_s3_uri=S3_FILEPATH,
#)

#TEST_SET_S3_URI = sagemaker.s3.S3Uploader.upload(
#    local_path=str(DATASET_FOLDER / "mnist_test.csv"), 
#    desired_s3_uri=S3_FILEPATH,
#)
#*************************************************************

#BUCKET = "mlschool-cda"
#S3_LOCATION = f"s3://{BUCKET}/penguins"
#DATA_FILEPATH = Path().resolve() / "data.csv"


sagemaker_client = boto3.client("sagemaker")
iam_client = boto3.client("iam")
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()

Overwriting codemnist/constantsmnist.py
